In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *
from setting import *

In [ ]:
selected_signal_matrix = pd.read_table(
    PATH_DICT["context_signal_matrix_file_path"], index_col=0
).fillna(0)

In [ ]:
k_nmfcc = ccal.nmf_consensus_cluster_with_multiple_k(
    selected_signal_matrix,
    NMF_KS,
    n_job=MAX_N_JOB,
    plot_df=False,
    plot_w=selected_signal_matrix.shape[0] < 3200,
    plot_h=selected_signal_matrix.shape[1] < 3200,
    directory_path=PATH_DICT["nmf_directory_path"],
)

In [ ]:
w = k_nmfcc["K{}".format(NMF_K)]["w"]

h = k_nmfcc["K{}".format(NMF_K)]["h"]

In [ ]:
for w_or_h, w_or_h_matrix in (("w", w), ("h", h)):

    if w_or_h_matrix.shape[1] < 3200:

        if w_or_h == "w":

            normalization_axis = 1

            cluster_axis = 0

        elif w_or_h == "h":

            normalization_axis = 0

            cluster_axis = 1

        ccal.plot_heat_map(
            w_or_h_matrix,
            normalization_axis=normalization_axis,
            normalization_method="-0-",
            cluster_axis=cluster_axis,
            title="NMF K={} {}".format(NMF_K, w_or_h.title()),
            xaxis_title=w_or_h_matrix.columns.name,
            yaxis_title=w_or_h_matrix.index.name,
        )

In [ ]:
for w_or_h, w_or_h_matrix in (("w", w.T), ("h", h)):

    title = "NMF K={} {}".format(NMF_K, w_or_h.title())

    ccal.make_comparison_panel(
        w_or_h_matrix,
        w_or_h_matrix,
        axis=1,
        title=title,
        name_0=w_or_h_matrix.index.name,
        name_1=w_or_h_matrix.index.name,
    )

In [ ]:
for w_or_h, w_or_h_df in (("w", w.T), ("h", h)):

    for p, values in w_or_h_df.iterrows():

        ccal.select_series_low_and_high_index(
            values,
            low_margin_factor=2,
            high_margin_factor=2,
            title="NMF K={} {} {} Signature".format(NMF_K, w_or_h.title(), p),
            file_path_prefix="{}/{}".format(
                PATH_DICT["{}_signature_directory_path".format(w_or_h)], p
            ),
        )

In [ ]:
for w_or_h, target_x_sample, features, directory_path in (
    ("w", w.T, selected_signal_matrix.T, PATH_DICT["w_match_directory_path"]),
    ("h", h, selected_signal_matrix, PATH_DICT["h_match_directory_path"]),
):

    ccal.make_match_panels(
        target_x_sample,
        {
            "Selected Signal": {"df": features, "data_type": "continuous"},
            "Input (Feature-by-Sample)": {
                "df": pd.read_table(FEATURE_X_SAMPLE_FILE_PATH, index_col=0),
                "data_type": "continuous",
            },
        },
        n_job=MAX_N_JOB,
        n_required_for_match_function=0.64,
        extreme_feature_threshold=EXTREME_FEATURE_THRESHOLD,
        plot_std=PLOT_STD,
        directory_path=directory_path,
    )